In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2

#lead pre-trained MobileNetV2
IMG_SIZE = (224, 224)
base_model= MobileNetV2(
    input_shape= IMG_SIZE + (3,),
    include_top= False,
    weights='imagenet'
)

print("MobileNetV2 loaded successfully")
print(f"Total layers in base model:{len(base_model.layers)}")


MobileNetV2 loaded successfully
Total layers in base model:154


In [8]:
#freeze all layers initially
for layer in base_model.layers:
  layer.trainable= False
#unfreeze n last layers for fine-tuning
N_LAST_LAYERS = 10
if N_LAST_LAYERS > 0:
  for layer in base_model.layers[-N_LAST_LAYERS:]:
    layer.trainable=True
#count trainable parameters
trainable_count= sum([tf.size(w).numpy() for w in base_model.trainable_weights])
non_trainable_count= sum([tf.size(w).numpy() for w in base_model.non_trainable_weights])

print(f"Trainable Parameters: {trainable_count:,}")
print(f"Non-Trainable Parameters: {non_trainable_count:,}")
print(f"Unfrozen last {N_LAST_LAYERS} layers for fine-tuning")



Trainable Parameters: 732,480
Non-Trainable Parameters: 1,525,504
Unfrozen last 10 layers for fine-tuning


In [9]:
#build complete model
inputs = keras.Input(shape=IMG_SIZE + (3,))

# Keep batchnorm layers frozen during training
x = base_model(inputs, training=False)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.35)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.25)(x)
NUM_CLASSES=38
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs, name="mobilenetv2_plant_disease_classifier")

#display model summary
model.summary()

Model: "mobilenetv2_plant_disease_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 38)             │         9,766 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,595,686 (9.90 MB)

 Trainable params: 1,070,182 (4.08 MB)

 Non-trainable params: 1,525,504 (5.82 MB)